# Домашнее задание к лекции «Понятие класса»
## Задание 1

Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса www.cbr-xml-daily.ru...ly_json.js


In [3]:
from libs.exchange import Rate

In [11]:
class Additions (Rate):
    def __init__(self, format_='value'):
        self.format = format_
    
    def find_max (self):
        response = self.exchange_rates()
        max=0
        for currency in response:
            if response[currency]['Value'] > max:
                max = response[currency]['Value']
                name = response[currency]['Name']
            else:
                continue
        print(f'Название валюты с максимальный значением курса - {name}, значение курса - {max}')

In [7]:
r = Additions()


In [8]:
r.find_max ()

Название валюты с максимальный значением курса - СДР (специальные права заимствования), значение курса - 104.1682


In [18]:
r.usd()

73.1909

## Задание 2

Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах курсов валют (eur, usd итд) будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

In [22]:
import requests

In [26]:
class Rate:
    def __init__(self, format_='value',diff=False):
        self.format = format_
        self.diff=diff
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return self.r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в трех вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        - частичная информация при self.format = 'value':
        self.diff = 'False' или пропуск - Значение курса валюты в текущий момент
        self.diff = 'True' - Разность курса валюты за текущий момент и предыдущий
        Rate('value').make_format('EUR')/Rate('value', False).make_format('EUR')
        79.4966
        Rate('value', True).make_format('EUR')
        0.31790000000000873
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                if self.diff==False:
                    return response[currency]['Value']
                if self.diff==True:
                    return (response[currency]['Value']-response[currency]['Previous'])
        
        return 'Error'
    
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
    
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')
    
    def brl(self):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL')

In [27]:
a=Rate('value',True)

In [28]:
a.eur()

0.31790000000000873